In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import base64
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################

username = "accuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('Brylene Patrick'))),
    html.Hr(),
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), id='my-image-brylene-patrick')
    ]),
    html.Div([
        html.Label('Select Rescue Type'),
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'}
            ],
            value='',
            clearable=True,
            style={'width': '50%'}
        ),
        html.Button('Reset Filters', id='reset-btn', n_clicks=0, style={'margin-left': '10px'}),
    ], style={'display': 'flex', 'justify-content': 'center', 'margin-top': '10px'}),

    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,          # disable user editing of cells
        filter_action='native',  # enable filtering of data
        sort_action='native',    # enable sorting of data
        sort_mode="multi",       # enable multi-column sorting
        row_selectable="multi",  # enable multiple row selection
        row_deletable=False,     # disable row deletion
        selected_columns=[],     # initially no columns selected
        page_action="native",    # enable pagination
        page_current=0,          # start on page 0
        page_size=10             # show 10 rows per page
        
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # to set up the filter query
    if filter_type == 'Water Rescue':
        filter_query = {'animal_type': 'Dog', 'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 'sex_upon_outcome': 'Intact Female', '$or': [{'age_upon_outcome_in_weeks': {'$gte': 26}}, {'age_upon_outcome_in_weeks': {'$lte': 156}}]}
    elif filter_type == 'Mountain or Wilderness Rescue':
        filter_query = {'animal_type': 'Dog', 'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}, 'sex_upon_outcome': 'Intact Male', '$or': [{'age_upon_outcome_in_weeks': {'$gte': 26}}, {'age_upon_outcome_in_weeks': {'$lte': 156}}]}
    elif filter_type == 'Disaster or Individual Tracking':
        filter_query = {'animal_type': 'Dog', 'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}, 'sex_upon_outcome': 'Intact Male', '$or': [{'age_upon_outcome_in_weeks': {'$gte': 20}}, {'age_upon_outcome_in_weeks': {'$lte': 300}}]}
    else:
        filter_query = {}
    
    # to retrieve the data from the database and convert it to a pandas dataframe
    animal_data = list(shelter.read(filter_query))
    df = pd.DataFrame(animal_data)

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # to create a pie chart showing the distribution of outcomes
    df = pd.DataFrame(viewData)
    outcome_counts = df['outcome_type'].value_counts()
    fig = px.pie(outcome_counts, values=outcome_counts.values, names=outcome_counts.index)
    return dcc.Graph(figure=fig)

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    # To create a dataframe from the viewData and set the animal type as the tooltip
    dff = pd.DataFrame.from_dict(viewData)
    # To set up the map
    map = dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
    dl.TileLayer(id="base-layer-id")
    ])
    # To add markers to the map
    for index, row in dff.iterrows():
        map.children.append(dl.Marker(position=[row['location_lat'], row['location_long']], children=[
        dl.Tooltip(row['breed']),
        dl.Popup([
        html.H1("Animal Name"),
        html.P(row['animal_id'])
            ])
        ]))
    return map

                               

app